Projeto de Verão
==
Introdução a Python
--
Feito por: Silvana Barboza

In [2]:
import ccxt
import datetime
from sqlalchemy import create_engine
import sqlite3 
import poloniex

### Definindo as variáveis


In [1]:
symbol1 = 'BTC_MAID'
symbol2 = 'BTC_STR'

### Pegando os valores


In [5]:
now = datetime.datetime.now()
yearAgo = now - datetime.timedelta(days=365)

In [6]:
data1 = poloniex.get_ohlc(symbol1, yearAgo, now)
data2 = poloniex.get_ohlc(symbol2, yearAgo, now)

### Salvando no banco de dados


In [10]:
conn = create_engine('sqlite:///summer_project.sqlite')

In [12]:
data1.to_sql(symbol1, conn, if_exists='replace')
data2.to_sql(symbol2, conn, if_exists='replace')

# Gráficos

In [7]:
#---imports---#
import holoviews as hv
from holoviews.operation.timeseries import rolling
import numpy as np
import pandas as pd
hv.extension('matplotlib')
hv.Dimension.type_formatters[np.datetime64] = '%m/%y'
%output fig='svg' dpi=300

In [8]:
##Opts
plot_opts = dict(fig_size=200, legend_position='top_left')
color_cycle = hv.Cycle(values=['#4169E1', '#2E8B57'])
hv.Store.options(backend='matplotlib').Curve = hv.Options('style', color=color_cycle)

#### Pegar valores do banco de dados


In [9]:
conn = create_engine('sqlite:///summer_project.sqlite')
data1 = pd.read_sql(symbol1, conn)
data2 = pd.read_sql(symbol2, conn)

## Gráfico 1
### Relação dos valores de fechamento entre duas moedas 


In [10]:
curve1 = hv.Curve(data1, ('date', 'Date'), ('close', 'Price'), label=symbol1)
curve2 = hv.Curve(data2, ('date', 'Date'), ('close', 'Price'), label=symbol2)

(curve1 * curve2).opts(plot=plot_opts)

:Overlay
   .Curve.BTC_MAID :Curve   [date]   (close)
   .Curve.BTC_STR  :Curve   [date]   (close)

## Gráfico 2
### Visualização dos maiores valores da moeda durante o mês

In [26]:
def get_highest_value_by_month(dataframe):
    df = pd.DataFrame(dataframe)
    df['month'] = df.date.dt.to_period('M')
    groupDf = df.groupby(['month'], sort=False, as_index=False)['high'].max()
    groupDf['date'] = groupDf['month'].map(lambda x: x.to_timestamp())
    return groupDf


In [27]:
def get_highest_value_by_day(dataframe):
    df = pd.DataFrame(dataframe)
    df['day'] = df.date.dt.to_period('d')
    groupDf = df.groupby(['day'], sort=False, as_index=False)['high'].max()
    groupDf['date'] = groupDf['day'].map(lambda x: x.to_timestamp())
    return groupDf

In [95]:
curve1 = hv.Curve(get_highest_value_by_month(data1), ('date', 'Date'), ('high', 'Price'), label=symbol1)
bars1 = hv.Bars(curve1)

curve2 = hv.Curve(get_highest_value_by_month(data2), ('date', 'Date'), ('high', 'Price'), label=symbol2)
bars2 = hv.Bars(curve2)

print("Maior valor da moeda mensal")
(bars1 + bars2).opts(plot=plot_opts)

Maior valor da moeda mensal


:Layout
   .Bars.BTC_MAID :Bars   [date]   (high)
   .Bars.BTC_STR  :Bars   [date]   (high)

## Gráfico 3
### Relação entre o maior e o menor preço de venda no mês

In [29]:
def get_lowest_value_by_month(dataframe):
    df = pd.DataFrame(dataframe)
    df['month'] = df.date.dt.to_period('M')
    groupDf = df.groupby(['month'], sort=False, as_index=False)['low'].min()
    groupDf['date'] = groupDf['month'].map(lambda x: x.to_timestamp())
    return groupDf

In [60]:
list = []
list.append(hv.Area(hv.Curve(get_highest_value_by_month(data1), 'date', 'high', label=symbol1)))
list.append(hv.Area(hv.Curve(get_lowest_value_by_month(data1), 'date', 'low', label=symbol1)))

list.append(hv.Area(hv.Curve(get_highest_value_by_month(data2), 'date', 'high', label=symbol2)))
list.append(hv.Area(hv.Curve(get_lowest_value_by_month(data2), 'date', 'low', label=symbol2)))

In [65]:
print("Gráfico do " + str(symbol1))
(list[0] * list[1]).opts(plot=plot_opts)

Gráfico do BTC_MAID


:Overlay
   .Area.BTC_MAID.I  :Area   [date]   (high)
   .Area.BTC_MAID.II :Area   [date]   (low)

In [66]:
print("Gráfico do " + str(symbol2))
(list[2] * list[3]).opts(plot=plot_opts)

Gráfico do BTC_STR


:Overlay
   .Area.BTC_STR.I  :Area   [date]   (high)
   .Area.BTC_STR.II :Area   [date]   (low)

## Gráfico 4
### Volume de Diário

In [53]:
def get_sum_volume_by_day(dataframe):
    df = pd.DataFrame(dataframe)
    df['day'] = df.date.dt.to_period('d')
    groupDf = df.groupby(['day'], sort=False, as_index=False)['volume'].sum()
    groupDf['date'] = groupDf['day'].map(lambda x: x.to_timestamp())
    return groupDf


In [42]:
def get_volume_by_hour(dataframe):
    df = pd.DataFrame(dataframe)
    df['hour'] = df.date.dt.to_period('h')
    print(df.date.dt.to_period('h'))
    groupDf = df.groupby(['hour'], sort=False, as_index=False)['volume']
    groupDf['date'] = groupDf['hour'].map(lambda x: x.to_timestamp())
    return groupDf

In [50]:
def get_one_month_data(dataframe):
    today = datetime.datetime.today()
    monthAgo = now - datetime.timedelta(days=30)
    dataframe.index = dataframe['date']
    return dataframe[monthAgo: today]   

In [59]:
graph4_opts = dict(fig_size=200, legend_position='top_left')

curveSumVolume1 = hv.Curve(get_sum_volume_by_day(data1), 'date', 'volume', label=symbol1).opts(plot=graph4_opts)
curveSumVolume2= hv.Curve(get_sum_volume_by_day(data2), 'date', 'volume', label=symbol2).opts(plot=graph4_opts)

curveSumVolume1 + curveSumVolume2

:Layout
   .Curve.BTC_MAID :Curve   [date]   (volume)
   .Curve.BTC_STR  :Curve   [date]   (volume)

## Gráfico 5
### Hora do menor valor do dia dos últimos 30 dias

In [79]:
def get_lowest_value_by_day(dataframe):
    df = pd.DataFrame(dataframe)
    df['day'] = df.date.dt.to_period('d')
    groupDf = df.groupby(['day'], sort=False)['low'].max()
    df['count_max'] = df.groupby(['day'])['low'].transform(max)
    df['time'] = df.date.dt.time
    return df[df['low'] == df['count_max']]

In [105]:
lowest1 = get_lowest_value_by_day(get_one_month_data(data1))
lowest2 = get_lowest_value_by_day(get_one_month_data(data2))

scatterOpts = dict(fig_size=200, legend_position='top_left', xrotation=360)
(hv.Scatter(lowest1, 'date', 'time', label=symbol1)).opts(plot=scatterOpts)

:Scatter   [date]   (time)